# Numerical Analysis Assignment #4

2018112749 전현승


## 1. 

In [1]:
import random
import numpy as np
import math

In [2]:
m = 3
b = 2

In [3]:
x = random.sample(range(-100, 101), 100)
print(x)
print(len(x))

[6, 3, -9, 33, -14, 31, -79, -20, 93, 54, -53, -54, 47, 53, -11, -85, -69, -43, 96, -83, -67, -52, 10, 75, 84, 82, -30, -2, -13, -84, -23, 66, 72, -95, -45, 38, -48, -49, -28, 64, 24, 17, 62, -8, 29, -18, -16, -12, -92, 58, 43, -39, -63, 39, -29, 27, 57, -22, 22, 30, 70, 4, -6, 68, -34, -68, -66, -27, 25, 73, -80, 76, 60, 63, -42, -93, -75, 44, 94, -1, -58, 56, 81, -40, 9, -98, -26, -61, 50, 48, -36, 80, 91, -10, -88, -62, 15, -19, -60, 32]
100


In [4]:
y = []
for i in range(0, 100):
  y.append(m * x[i] + b + np.random.normal(0, 2))

print(y)
print(len(y))

[17.722704571067972, 8.752359412555053, -25.574353212069163, 101.31434477249144, -39.784179862959554, 93.26221038785762, -236.72797552828183, -59.70330234290258, 281.9237340273431, 168.03878175631192, -157.235808773737, -158.3680148514864, 143.55199825470035, 161.28764600328503, -32.44646141548491, -254.168198102123, -204.2924615115175, -122.10542288119143, 289.4827063488513, -246.06555266325125, -196.26537833099488, -152.01153791061222, 36.346316546769664, 227.8922851442501, 252.57092621663796, 248.97956497440614, -86.69874531247036, -6.690386301731776, -36.99055217626922, -252.73969623172815, -67.8895107356638, 202.21880191458868, 217.00472614700442, -282.143182310107, -132.7750045962067, 114.30518572831595, -139.0718978197831, -145.8717289171963, -82.02886945347309, 196.58837149391718, 72.19063196566927, 53.83627658308423, 189.08078932034795, -22.50725274978041, 89.44162521672081, -54.17663416486939, -46.76493431034462, -38.81892282724724, -275.121971028747, 177.1406617302655, 133.3

## 2. Total least square fit

In [5]:
x_bar = np.average(x)
y_bar = np.average(y)

m_estimate_numerator = 0
m_estimate_denominator = 0

for i in range(0, 100):
  m_estimate_numerator += (x[i] - x_bar) * (y[i] - y_bar)
  m_estimate_denominator += (x[i] - x_bar) ** 2

m_estimate = m_estimate_numerator / m_estimate_denominator
b_estimate = y_bar - m_estimate * x_bar

print(f"Estimated line equation: y={m_estimate}x+{b_estimate}")

Estimated line equation: y=3.005696640094814x+2.174992622186176


In [6]:
MSE = 0
for i in range(0, 100):
  MSE += (y[i] - (m_estimate * x[i] + b_estimate)) ** 2
MSE /= 100

print(f"MSE: {MSE}")

MSE: 3.579020939954454


## 3. RANSAC

In [7]:
iteration = 200
dist_t = 2

In [8]:
res_m = 0
res_b = 0
res_points_x = []
res_points_y = []

for i in range(0, iteration):
  # Randomly select minimal subset of points
  idx1 = 0
  idx2 = 0
  while idx1 == idx2:
    idx1, idx2 = random.sample(range(0, 100), 2)
  x1, y1, x2, y2 = x[idx1], y[idx1], x[idx2], y[idx2]
  # print(x1, y1, x2, y2)

  # Hypothesize a model
  # ax+by+c=0
  # converted to by=-ax-c later
  a = (y1 - y2) / (x2 - x1)
  b = 1
  c = (x1 * (y2 - y1)) / (x2 - x1) - y1
  
  # Compute error function
  points_x = []
  points_y = []
  for j in range(0, 100):
    # (x[j], y[j])
    dist = abs(a*x[j] + b*y[j] + c) / math.sqrt(a**2 + b**2)
    # print(dist)
    if dist < dist_t:
      points_x.append(x[j])
      points_y.append(y[j])

  if len(res_points_x) < len(points_x):
    print(f"renewed: {len(res_points_x)} < {len(points_x)}")
    print(f"res_m: {res_m}, res_b: {res_b}")
    res_points_x = points_x[:]
    res_points_y = points_y[:]
    res_m = -a
    res_b = -c


renewed: 0 < 12
res_m: 0, res_b: 0
renewed: 12 < 96
res_m: 2.629701044014568, res_b: -30.64360936088471
renewed: 96 < 100
res_m: 3.0330895849686, res_b: 0.4189783645654117


In [9]:
print(f"Estimated line equation: y={res_m}x+{res_b}")

MSE = 0
for i in range(0, 100):
  MSE += (y[i] - (res_m * x[i] + res_b)) ** 2
MSE /= 100

print(f"MSE: {MSE}")

Estimated line equation: y=3.0010886565227746x+2.8224889707250327
MSE: 4.065322724300733


## 4. Compare two parameter estimation methods.

- Total least square fit 방법으로는 MSE가 4.581266008332804 정도, RANSAC 방법으로는 MSE가 4.814118890356541 정도로 도출되었다. 결과만 보면 Total least square fit 방법이 더 정확한 결과가 도출되었지만, 두 방법을 두고 여러 번 테스트해본다면 차이를 더 정확히 파악할 수 있을 것이다.